In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from eventlet.timeout import Timeout
import random
import time

# import auxillary packages
import requests  # for loading the example source code
import openai
import json

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from flaml.autogen.code_utils import extract_code
from create_bench_agent import CreateBenchAgent



In [2]:
example_qa_user = """
----------
Param Update: ```JSON
"end_charge_time": 9```
Example User request: Charge the car till 9 AM.
----------
Param Update: ```JSON
"required_charge": 100```
Example Request: Charge at least 100 KWH.
----------
Param Update: ```JSON
"max_charge": 150```
Example Request: Charge at most 150 KWH.
"""

In [3]:
example_qa_mgr = """
----------
Param Update: ```JSON
"carbon_cost": 20```
Example User request: Change carbon cost to 20.
----------
Param Update: ```JSON
"carbon_cost_weight": 0.9```
Example Request: Change carbon cost weight to 0.9.
----------
Param Update: ```JSON
"max_power": 150```
Example Request: Change max power to 150.
"""

In [4]:
desc_filepath = "values_desc.json"

In [5]:
with open(desc_filepath, 'r') as f:
    desc_json_str = f.read()

In [6]:
mgr_json_str = json.dumps(json.loads(desc_json_str)['values_lot_mgr'])

In [7]:
car_json_str = json.dumps(json.loads(desc_json_str)['values_car'])

In [8]:
log_history = {}

In [9]:
autogen.oai.ChatCompletion.start_logging(log_history)

In [10]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": [ "gpt-4"],
    },
)

In [11]:
agent_mgr = CreateBenchAgent(
    name = "CreateBenchAgent",
    example_qa=example_qa_mgr,
    json_str=mgr_json_str,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

In [12]:
agent_user = CreateBenchAgent(
    name = "CreateBenchAgent",
    example_qa=example_qa_user,
    json_str=car_json_str,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

In [13]:
with open('benchmark_values.json', 'r') as f:
    # text = f.read()
    values_db = json.loads(f.read())

In [14]:
values_mgr = values_db['values_lot_mgr']
values_car = values_db['values_car']

In [15]:
def getRange(args):
    return [round(x * args[3], 2) for x in np.arange(args[0], args[1], args[2], dtype=int)]


# Manager

In [16]:
param_update_single = """```JSON
"{key}": {new_val}
```"""

In [17]:
benchmark_mgr = []
num_elems = 20
for key, val_range in values_mgr.items():
    val_list = random.sample(list(getRange(val_range)), min(num_elems, len(list(getRange(val_range)))))
    for new_val in val_list:
        benchmark_instance = {}
        benchmark_instance["json_str"] = param_update_single.format(key = key, new_val = new_val)
        
        log_history = {}
        autogen.oai.ChatCompletion.start_logging(log_history)
        
        user.initiate_chat(agent_mgr, message = benchmark_instance["json_str"], silent = True, clear_history = True)

        log_list = []
        log_list = eval(list(log_history.keys())[-1])
        benchmark_instance['prompt'] = ""+(log_list[-1]['content'])

        benchmark_mgr.append(benchmark_instance)
        if len(benchmark_mgr) == 0 or (len(benchmark_mgr) > 0 and benchmark_mgr[-1]['prompt'] != benchmark_instance['prompt']):
            benchmark_mgr.append(benchmark_instance)
        
        time.sleep(0.5)
        print(len(benchmark_mgr))

CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Change carbon cost to 40.

--------------------------------------------------------------------------------
1
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Change carbon cost to 25.

--------------------------------------------------------------------------------
2
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Change carbon cost to 35.

--------------------------------------------------------------------------------
3
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenc

In [18]:
benchmark_combined_mgr = benchmark_mgr.copy()

In [19]:
benchmark_mgr_dataset_filename = "EV_single_mgr.benchmark.json"
with open(benchmark_mgr_dataset_filename, "w") as text_file:
    text_file.write(json.dumps(benchmark_mgr, indent=2))

In [20]:
param_update_multiple = """```JSON
"{key1}": {val1},
"{key2}": {val2}
```"""

In [21]:
params_to_change = list(values_mgr.keys())
num_elems = 7

In [22]:
benchmark_mgr = []
for i in range(len(params_to_change)):
    for j in range(i+1, len(params_to_change)):
        key1 = params_to_change[i]
        key2 = params_to_change[j]
        
        val_range1 = values_mgr[key1]
        val_range2 = values_mgr[key2]

        val_list1 = random.sample(list(getRange(val_range1)), min(num_elems, len(list(getRange(val_range1)))))
        val_list2 = random.sample(list(getRange(val_range2)), min(num_elems, len(list(getRange(val_range2)))))
        for new_val1 in val_list1:
            for new_val2 in val_list2:
                benchmark_instance = {}
                benchmark_instance["json_str"] = param_update_multiple.format(key1 = key1, 
                                                                            val1 = new_val1,
                                                                            key2 = key2,
                                                                            val2 = new_val2)
                
                log_history = {}
                autogen.oai.ChatCompletion.start_logging(log_history)

                user.initiate_chat(agent_mgr, message = benchmark_instance["json_str"], 
                                            silent = True, clear_history = True)

                log_list = eval(list(log_history.keys())[-1])
                benchmark_instance['prompt'] = ""+(log_list[-1]['content'])
                
                if len(benchmark_mgr) == 0 or (len(benchmark_mgr) > 0 and benchmark_mgr[-1]['prompt'] != benchmark_instance['prompt']):
                    benchmark_mgr.append(benchmark_instance)
                
                time.sleep(0.5)
                print(len(benchmark_mgr))

CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Change carbon cost to 5 and carbon cost weight to 0.55.

--------------------------------------------------------------------------------
1
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Update the carbon emissions per KWH each hour of day to 5 and set the total cost's weightage coming from carbon cost to 0.85.

--------------------------------------------------------------------------------
2
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Change the carbon emissions per KWH each hour of day to 5 and adjust the carbon cost weight to 0.1.

---------------------------------------

In [23]:
benchmark_dataset_filename = "EV_multiple_mgr.benchmark.json"
with open(benchmark_dataset_filename, "w") as text_file:
    text_file.write(json.dumps(benchmark_mgr, indent=2))

In [24]:
benchmark_combined_mgr.extend(benchmark_mgr)

In [25]:
i = 0
for benchmark_instance in benchmark_combined_mgr:
    benchmark_instance['index'] = i
    i = i + 1


In [26]:
benchmark_dataset_filename = "EV_combined_mgr.benchmark.json"
with open(benchmark_dataset_filename, "w") as text_file:
    text_file.write(json.dumps(benchmark_combined_mgr, indent=2))

# Single car

In [27]:
benchmark_car = []
num_elems = 20
for key, val_range in values_car.items():
    val_list = random.sample(list(getRange(val_range)), min(num_elems, len(list(getRange(val_range)))))
    for new_val in val_list:
        benchmark_instance = {}
        benchmark_instance["json_str"] = param_update_single.format(key = key, new_val = new_val)
        
        log_history = {}
        autogen.oai.ChatCompletion.start_logging(log_history)
        
        user.initiate_chat(agent_user, message = benchmark_instance["json_str"], silent = True, clear_history = True)

        log_list = []
        log_list = eval(list(log_history.keys())[-1])
        benchmark_instance['prompt'] = ""+(log_list[-1]['content'])

        benchmark_car.append(benchmark_instance)
        if len(benchmark_car) == 0 or (len(benchmark_car) > 0 and benchmark_car[-1]['prompt'] != benchmark_instance['prompt']):
            benchmark_car.append(benchmark_instance)
        
        time.sleep(0.5)
        print(len(benchmark_car))

CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Charge my car with at least 30 KWH.

--------------------------------------------------------------------------------
1
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Charge at least 35 KWH.

--------------------------------------------------------------------------------
2
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Charge at least 85 KWH.

--------------------------------------------------------------------------------
3
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to Crea

In [28]:
benchmark_combined_car = benchmark_car.copy()

In [29]:
benchmark_car_dataset_filename = "EV_single_car.benchmark.json"
with open(benchmark_car_dataset_filename, "w") as text_file:
    text_file.write(json.dumps(benchmark_car, indent=2))

In [30]:
param_update_multiple = """```JSON
"{key1}": {val1},
"{key2}": {val2}
```"""

In [31]:
params_to_change = list(values_car.keys())
num_elems = 7

In [32]:
benchmark_car = []
for i in range(len(params_to_change)):
    for j in range(i+1, len(params_to_change)):
        key1 = params_to_change[i]
        key2 = params_to_change[j]
        
        val_range1 = values_car[key1]
        val_range2 = values_car[key2]

        val_list1 = random.sample(list(getRange(val_range1)), min(num_elems, len(list(getRange(val_range1)))))
        val_list2 = random.sample(list(getRange(val_range2)), min(num_elems, len(list(getRange(val_range2)))))
        for new_val1 in val_list1:
            for new_val2 in val_list2:
                benchmark_instance = {}
                benchmark_instance["json_str"] = param_update_multiple.format(key1 = key1, 
                                                                            val1 = new_val1,
                                                                            key2 = key2,
                                                                            val2 = new_val2)
                
                log_history = {}
                autogen.oai.ChatCompletion.start_logging(log_history)

                user.initiate_chat(agent_user, message = benchmark_instance["json_str"], 
                                            silent = True, clear_history = True)

                log_list = eval(list(log_history.keys())[-1])
                benchmark_instance['prompt'] = ""+(log_list[-1]['content'])
                
                if len(benchmark_car) == 0 or (len(benchmark_car) > 0 and benchmark_car[-1]['prompt'] != benchmark_instance['prompt']):
                    benchmark_car.append(benchmark_instance)
                
                time.sleep(0.5)
                print(len(benchmark_car))

CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Charge my car with at least 55 KWH, but do not exceed 70 KWH.

--------------------------------------------------------------------------------
1
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Request a required charge of 55 KWH but do not exceed a maximum of 40 KWH.

--------------------------------------------------------------------------------
2
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Charge at least 55 KWH, but not more than 30 KWH.

--------------------------------------------------------------------------------
3
CreateBenchAgent (to writer):




Describe the chan

In [33]:
benchmark_dataset_filename = "EV_multiple_car.benchmark.json"
with open(benchmark_dataset_filename, "w") as text_file:
    text_file.write(json.dumps(benchmark_car, indent=2))

In [34]:
benchmark_combined_car.extend(benchmark_car)

In [35]:
i = 0
for benchmark_instance in benchmark_combined_car:
    benchmark_instance['index'] = i
    i = i + 1


In [36]:
benchmark_dataset_filename = "EV_combined_car.benchmark.json"
with open(benchmark_dataset_filename, "w") as text_file:
    text_file.write(json.dumps(benchmark_combined_car, indent=2))

In [37]:
len(benchmark_combined_car)

207

In [38]:
benchmark_car

[{'json_str': '```JSON\n"required_charge": 55,\n"max_charge": 70\n```',
  'prompt': 'Charge my car with at least 55 KWH, but do not exceed 70 KWH.',
  'index': 60},
 {'json_str': '```JSON\n"required_charge": 55,\n"max_charge": 40\n```',
  'prompt': 'Request a required charge of 55 KWH but do not exceed a maximum of 40 KWH.',
  'index': 61},
 {'json_str': '```JSON\n"required_charge": 55,\n"max_charge": 30\n```',
  'prompt': 'Charge at least 55 KWH, but not more than 30 KWH.',
  'index': 62},
 {'json_str': '```JSON\n"required_charge": 55,\n"max_charge": 75\n```',
  'prompt': "Increase the requested charge for the car to 55 KWH and don't exceed the supply more than 75 KWH.",
  'index': 63},
 {'json_str': '```JSON\n"required_charge": 55,\n"max_charge": 0\n```',
  'prompt': 'Charge a minimum of 55 KWH.',
  'index': 64},
 {'json_str': '```JSON\n"required_charge": 55,\n"max_charge": 65\n```',
  'prompt': 'Charge with a minimum of 55 KWH and a maximum of 65 KWH.',
  'index': 65},
 {'json_str':